In [2]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import sys 

In [3]:
# # Connect Google Drive to a Google Colab Notebook for Dataset
# from google.colab import drive

# drive.mount('/content/gdrive/', force_remount=True)

In [4]:
def is_path_exists(path):
    if(os.path.exists(path)):
        return True
    else:
        return False

In [5]:
def ls(path):
    """
    path: path of required directory, want to know name of sub-directories
    """
    if (is_path_exists(path)):
        return os.listdir(path)

In [6]:
dataset_path = 'D:\\Dataset\\Rose_lab\\processed_data\\required_dataset_skeleton_combined_with_classes\\'

In [7]:
ls(dataset_path)

['A041',
 'A042',
 'A043',
 'A044',
 'A045',
 'A046',
 'A047',
 'A048',
 'A049',
 'A103',
 'A104',
 'A105',
 'npy_dataset',
 'npy_ntu_rgb120_missings.txt']

In [8]:
req_classes = ['A049', 'A044', 'A046', 'A105', 'A043', 'A045', 'A103', 'A104', 'A047', 'A048', 'A041', 'A042']
req_classes.sort()
print(req_classes)

['A041', 'A042', 'A043', 'A044', 'A045', 'A046', 'A047', 'A048', 'A049', 'A103', 'A104', 'A105']


In [9]:
for r in req_classes:
    files_name = ls("{}\\{}".format(dataset_path,  r))
    print(len(files_name))

948
948
948
948
948
948
948
948
948
960
960
960


In [10]:
def req_directory_maker():
    if not os.path.exists(dataset_path + 'npy_dataset'):
        os.mkdir(dataset_path + 'npy_dataset')
        os.chdir(dataset_path + 'npy_dataset')
        f = open(dataset_path + 'npy_ntu_rgb120_missings.txt', 'w')
        f.close()
        for r in req_classes:
            os.mkdir(r)
    


def path_setter(req_class):
    user_name = 'content'
    save_npy_path = '{}{}\\{}\\'.format(dataset_path, 'npy_dataset', req_class)
    load_txt_path = '{}{}\\'.format(dataset_path, req_class)
    missing_file_path = '{}npy_ntu_rgb120_missings.txt'.format(dataset_path)
    step_ranges = list(range(0,1000)) # just parse range, for the purpose of paralle running. 

    return user_name, save_npy_path, load_txt_path, missing_file_path, step_ranges

In [11]:
toolbar_width = 50
def _print_toolbar(rate, annotation=''):
    sys.stdout.write("{}[".format(annotation))
    for i in range(toolbar_width):
        if i * 1.0 / toolbar_width > rate:
            sys.stdout.write(' ')
        else:
            sys.stdout.write('-')
        sys.stdout.flush()
    sys.stdout.write(']\r')

In [12]:
def _end_toolbar():
    sys.stdout.write('\n')

def _load_missing_file(path):
    missing_files = dict()
    with open(path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line[:-1]
            if line not in missing_files:
                missing_files[line] = True 
    return missing_files

In [13]:
def _read_skeleton(file_path, save_skelxyz=True, save_rgbxy=True, save_depthxy=True):
    f = open(file_path, 'r')
    datas = f.readlines()
    f.close()
    max_body = 4
    njoints = 25

    # specify the maximum number of the body shown in the sequence, according to the certain sequence, need to pune the 
    # abundant bodys. 
    # read all lines into the pool to speed up, less io operation. 
    nframe = int(datas[0][:-1])
    bodymat = dict()
    bodymat['file_name'] = file_path[-29:-9]
    nbody = int(datas[1][:-1])
    bodymat['nbodys'] = [] 
    bodymat['njoints'] = njoints 
    for body in range(max_body):
        if save_skelxyz:
            bodymat['skel_body{}'.format(body)] = np.zeros(shape=(nframe, njoints, 3))
        if save_rgbxy:
            bodymat['rgb_body{}'.format(body)] = np.zeros(shape=(nframe, njoints, 2))
        if save_depthxy:
            bodymat['depth_body{}'.format(body)] = np.zeros(shape=(nframe, njoints, 2))
    # above prepare the data holder
    cursor = 0
    for frame in range(nframe):
        cursor += 1
        bodycount = int(datas[cursor][:-1])    
        if bodycount == 0:
            continue 
        # skip the empty frame 
        bodymat['nbodys'].append(bodycount)
        for body in range(bodycount):
            cursor += 1
            skel_body = 'skel_body{}'.format(body)
            rgb_body = 'rgb_body{}'.format(body)
            depth_body = 'depth_body{}'.format(body)
            
            bodyinfo = datas[cursor][:-1].split(' ')
            cursor += 1
            
            njoints = int(datas[cursor][:-1])
            for joint in range(njoints):
                cursor += 1
                jointinfo = datas[cursor][:-1].split(' ')
                jointinfo = np.array(list(map(float, jointinfo)))
                if save_skelxyz:
                    bodymat[skel_body][frame,joint] = jointinfo[:3]
                if save_depthxy:
                    bodymat[depth_body][frame,joint] = jointinfo[3:5]
                if save_rgbxy:
                    bodymat[rgb_body][frame,joint] = jointinfo[5:7]
    # prune the abundant bodys 
    for each in range(max_body):
        if each >= max(bodymat['nbodys']):
            if save_skelxyz:
                del bodymat['skel_body{}'.format(each)]
            if save_rgbxy:
                del bodymat['rgb_body{}'.format(each)]
            if save_depthxy:
                del bodymat['depth_body{}'.format(each)]
    return bodymat

In [14]:
missing_file_path = '{}npy_ntu_rgb120_missings.txt'.format(dataset_path)
with open(missing_file_path, encoding = 'utf-8') as f:
    f = f.read()
f = f.split('\n')

required_classes = "A041|A042|A043|A044|A045|A046|A047|A048|A049|A103|A104|A105"
pattern = '.*({})'.format(required_classes)

missing_files_name = [x for x in f if re.match(pattern, x)]
missing_files_name

['S005C002P010R001A048',
 'S005C002P010R001A049',
 'S006C002P016R001A043',
 'S008C003P025R002A049',
 'S008C003P034R002A044',
 'S008C003P034R002A045',
 'S010C003P007R002A043',
 'S014C003P025R001A048',
 'S015C003P025R002A044',
 'S017C002P015R001A042',
 'S024C002P067R001A103',
 'S029C003P043R001A105',
 'S030C002P091R002A103',
 'S030C002P091R002A104',
 'S030C002P091R002A105',
 'S005C002P010R001A048',
 'S005C002P010R001A049',
 'S006C002P016R001A043',
 'S008C003P025R002A049',
 'S008C003P034R002A044',
 'S008C003P034R002A045',
 'S010C003P007R002A043',
 'S014C003P025R001A048',
 'S015C003P025R002A044',
 'S017C002P015R001A042']

In [56]:
if __name__ == '__main__':
    req_directory_maker()
    for r in req_classes:
        user_name, save_npy_path, load_txt_path, missing_file_path, step_ranges = path_setter(r)

        missing_files = _load_missing_file(missing_file_path)
        datalist = os.listdir(load_txt_path)
        alread_exist = os.listdir(save_npy_path)
        alread_exist_dict = dict(zip(alread_exist, len(alread_exist) * [True]))
     
        for ind, each in enumerate(datalist):
            _print_toolbar(ind * 1.0 / len(datalist),
                           '({:>5}/{:<5})'.format(
                               ind + 1, len(datalist)
                           ))
            S = int(each[1:4])
            if S not in step_ranges:
                continue 
            if each+'.skeleton.npy' in alread_exist_dict:
                print('file already existed !')
                continue
            if each[:20] in missing_files:
                print('file missing')
                continue 
            loadname = load_txt_path+each
            print(each)
            mat = _read_skeleton(loadname)
            mat = np.array(mat)
            save_path = save_npy_path+'{}.npy'.format(each)
            np.save(save_path, mat)
            # raise ValueError()
        _end_toolbar()
#     data_saver_google_drive()
#     shutil.rmtree(save_path)

S001C001P001R001A041.skeleton                                   ]
S001C001P001R002A041.skeleton                                   ]
S001C001P002R001A041.skeleton                                   ]
S001C001P002R002A041.skeleton                                   ]
S001C001P003R001A041.skeleton                                   ]
S001C001P003R002A041.skeleton                                   ]
S001C001P004R001A041.skeleton                                   ]
S001C001P004R002A041.skeleton                                   ]
S001C001P005R001A041.skeleton                                   ]
S001C001P005R002A041.skeleton                                   ]
S001C001P006R001A041.skeleton                                   ]
S001C001P006R002A041.skeleton                                   ]
S001C001P007R001A041.skeleton                                   ]
S001C001P007R002A041.skeleton                                   ]
S001C001P008R001A041.skeleton                                   ]
S001C001P0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



S003C001P007R002A041.skeleton             ]
S003C001P008R001A041.skeleton                                   ]
S003C001P008R002A041.skeleton                                   ]
S003C001P015R001A041.skeleton                                   ]
S003C001P015R002A041.skeleton                                   ]
S003C001P016R001A041.skeleton                                   ]
S003C001P016R002A041.skeleton                                   ]
S003C001P017R001A041.skeleton                                   ]
S003C001P017R002A041.skeleton                                   ]
S003C001P018R001A041.skeleton                                   ]
S003C001P018R002A041.skeleton                                   ]
S003C001P019R001A041.skeleton                                   ]
S003C001P019R002A041.skeleton                                   ]
S003C002P001R001A041.skeleton                                   ]
S003C002P001R002A041.skeleton                                   ]
S003C002P002R001A041.skeleton   

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



S005C003P016R002A041.skeleton             ]
S005C003P017R001A041.skeleton                                   ]
S005C003P017R002A041.skeleton                                   ]
S005C003P018R001A041.skeleton                                   ]
S005C003P018R002A041.skeleton                                   ]
S005C003P021R001A041.skeleton                                   ]
S005C003P021R002A041.skeleton                                   ]
S006C001P001R001A041.skeleton                                   ]
S006C001P001R002A041.skeleton                                   ]
S006C001P007R001A041.skeleton                                   ]
S006C001P007R002A041.skeleton                                   ]
S006C001P008R001A041.skeleton                                   ]
S006C001P008R002A041.skeleton                                   ]
S006C001P015R001A041.skeleton                                   ]
S006C001P015R002A041.skeleton                                   ]
S006C001P016R001A041.skeleton   

...

S030C003P048R002A105.skeleton
S030C003P067R001A105.skeleton--------------------------         ]
S030C003P067R002A105.skeleton--------------------------         ]
S030C003P075R001A105.skeleton--------------------------         ]
S030C003P075R002A105.skeleton--------------------------         ]
S030C003P076R001A105.skeleton--------------------------         ]
S030C003P076R002A105.skeleton--------------------------         ]
S030C003P081R001A105.skeleton--------------------------         ]
S030C003P081R002A105.skeleton--------------------------         ]
S030C003P085R001A105.skeleton---------------------------        ]
S030C003P085R002A105.skeleton---------------------------        ]
S030C003P088R001A105.skeleton---------------------------        ]
S030C003P088R002A105.skeleton---------------------------        ]
S030C003P091R001A105.skeleton---------------------------        ]
S030C003P091R002A105.skeleton---------------------------        ]
S030C003P092R001A105.skeleton-----------------

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



S031C002P067R001A105.skeleton
S031C002P067R002A105.skeleton------------------------------     ]
S031C002P082R001A105.skeleton------------------------------     ]
S031C002P082R002A105.skeleton------------------------------     ]
S031C002P085R001A105.skeleton------------------------------     ]
S031C002P085R002A105.skeleton------------------------------     ]
S031C002P096R001A105.skeleton------------------------------     ]
S031C002P096R002A105.skeleton------------------------------     ]
S031C002P097R001A105.skeleton------------------------------     ]
S031C002P097R002A105.skeleton------------------------------     ]
S031C002P098R001A105.skeleton------------------------------     ]
S031C002P098R002A105.skeleton------------------------------     ]
S031C002P099R001A105.skeleton------------------------------     ]
S031C002P099R002A105.skeleton------------------------------     ]
S031C002P100R001A105.skeleton-------------------------------    ]
S031C002P100R002A105.skeleton-----------------

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



S032C002P067R002A105.skeleton ]
S032C002P102R001A105.skeleton---------------------------------- ]
S032C002P102R002A105.skeleton---------------------------------- ]
S032C002P103R001A105.skeleton---------------------------------- ]
S032C002P103R002A105.skeleton---------------------------------- ]
S032C002P104R001A105.skeleton---------------------------------- ]
S032C002P104R002A105.skeleton---------------------------------- ]
S032C002P105R001A105.skeleton---------------------------------- ]
S032C002P105R002A105.skeleton---------------------------------- ]
S032C002P106R001A105.skeleton---------------------------------- ]
S032C002P106R002A105.skeleton---------------------------------- ]
S032C003P008R001A105.skeleton---------------------------------- ]
S032C003P008R002A105.skeleton-----------------------------------]
S032C003P041R001A105.skeleton-----------------------------------]
S032C003P041R002A105.skeleton-----------------------------------]
S032C003P043R001A105.skeleton---------------

In [ ]:
total = 0
for x in range(0, 41):
  count = 0
  pattern = ".*P00{}.*".format(x)
  if x >=10:
    pattern = ".*P0{}.*".format(x)
  for file in files_name:
    if re.match(pattern, file):
      count += 1
  total = total + count
  print("{}: {}".format(pattern, count))
print(len(files_name), total)


Setup No.
A043:<br>
^S001: 48<br>
^S002: 54<br>
^S003: 54<br>
^S004: 24<br>
^S005: 48<br>
^S006: 60<br>
^S007: 72<br>
^S008: 84<br>
^S009: 42<br>
^S010: 60<br>
^S011: 78<br>
^S012: 66<br>
^S013: 66<br>
^S014: 54<br>
^S015: 48<br>
^S016: 42<br>
^S017: 48<br>
948 948<br>

Camera Id:<br>
.*C001.*: 316<br>
.*C002.*: 316<br>
.*C003.*: 316<br>

performer id<br>
.*P001.*: 36<br>
.*P002.*: 18<br>
.*P003.*: 24<br>
.*P004.*: 12<br>
.*P005.*: 6<br>
.*P006.*: 6<br>
.*P007.*: 96<br>
.*P008.*: 96<br>
.*P009.*: 12<br>
.*P010.*: 12<br>
.*P011.*: 6<br>
.*P012.*: 6<br>
.*P013.*: 18<br>
.*P014.*: 6<br>
.*P015.*: 78<br>
.*P016.*: 66<br>
.*P017.*: 72<br>
.*P018.*: 42<br>
.*P019.*: 72<br>
.*P020.*: 12<br>
.*P021.*: 18<br>
.*P022.*: 6<br>
.*P023.*: 6<br>
.*P024.*: 6<br>
.*P025.*: 60<br>
.*P026.*: 6<br>
.*P027.*: 30<br>
.*P028.*: 24<br>
.*P029.*: 6<br>
.*P030.*: 6<br>
.*P031.*: 6<br>
.*P032.*: 6<br>
.*P033.*: 6<br>
.*P034.*: 6<br>
.*P035.*: 6<br>
.*P036.*: 6<br>
.*P037.*: 24<br>
.*P038.*: 6<br>
.*P039.*: 12<br>
.*P040.*: 6<br>
948 948<br>

In [ ]:
count

In [ ]:
files_name

In [ ]:
# %cd /content/gdrive/MyDrive/required_dataset_skeleton_combined_with_classes

In [ ]:
# !ls

In [ ]:
# %cd A041

In [ ]:
# with open("S001C001P001R001A041.skeleton", 'r') as f:
  # print(f)

In [15]:
import numpy as np

In [23]:
mat = np.arange(0,18).reshape(2,3,3)

In [24]:
mat

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]]])

In [25]:
mat[0]

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [42]:
np.subtract(mat,mat[0][0])

array([[[ 0,  0,  0],
        [ 3,  3,  3],
        [ 6,  6,  6]],

       [[ 9,  9,  9],
        [12, 12, 12],
        [15, 15, 15]]])

In [43]:
mat

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]]])

In [51]:
mat[:,0]

array([[ 0,  1,  2],
       [ 9, 10, 11]])

In [57]:
mat[1]

array([[ 9, 10, 11],
       [12, 13, 14],
       [15, 16, 17]])

In [58]:
mat[:,0][1]

array([ 9, 10, 11])

In [61]:
mat[0][1:] - mat[:,0][0]

array([[3, 3, 3],
       [6, 6, 6]])